# impoort Libraries

In [1]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
%matplotlib inline
import os
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.models import resnet18
import random
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.1 MB/s eta 0:00:00


In [3]:
import timm

# Loading of CIFAR-10 Dataset with Data Transformations

In [4]:
noise_levels = [0.1, 0.3, 0.5, 0.8, 0.9]

# Define class names
class_names = [
    'airplane', 'automobile', 'bird', 'cat', 'deer',
    'dog', 'frog', 'horse', 'ship', 'truck'
]

# Define transformations
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

100%|██████████| 170498071/170498071 [00:01<00:00, 95006887.99it/s] 


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


#Implementation of SCELoss and Label Noise Functions

In [5]:
# SCELoss class definition
class SCELoss(torch.nn.Module):
    def __init__(self, alpha, beta, num_classes=10):
        super(SCELoss, self).__init__()
        self.alpha = alpha  # Weight for the cross-entropy loss component
        self.beta = beta  # Weight for the reverse cross-entropy loss component
        self.num_classes = num_classes  # Number of classes in the dataset
        self.cross_entropy = torch.nn.CrossEntropyLoss()  # Standard cross-entropy loss

    def forward(self, pred, labels):
        ce = self.cross_entropy(pred, labels)  # Calculate the standard cross-entropy loss
        pred = F.softmax(pred, dim=1)  # Apply softmax to predictions
        pred = torch.clamp(pred, min=1e-7, max=1.0)  # Clamp values to avoid log(0)
        label_one_hot = F.one_hot(labels, self.num_classes).float()  # Convert labels to one-hot encoding
        label_one_hot = torch.clamp(label_one_hot, min=1e-4, max=1.0)  # Clamp values for stability
        rce = (-1 * torch.sum(pred * torch.log(label_one_hot), dim=1))  # Calculate reverse cross-entropy
        loss = self.alpha * ce + self.beta * rce.mean()  # Combine the two components of the loss
        return loss

    # Function to apply symmetric label noise
def symmetric_label_noise(labels, epsilon):
    noisy_labels = []
    for label in labels:
        if random.random() < epsilon:
            # With probability epsilon, pick a random label different from the true label
            noisy_label = random.choice([i for i in range(10) if i != label])
            noisy_labels.append(noisy_label)
        else:
            noisy_labels.append(label)  # With remaining probability, keep the original label
    return noisy_labels

def asymmetric_label_noise( labels, epsilon):
    noisy_labels = []
    for label in labels:
       # Apply label-specific noise based on the predefined noise pattern
        if label == 9:  # Check if the label is 'bird'
            if random.random() < epsilon:
                noisy_label = 0  # Flip 'bird' to 'airplane'
            else:
                noisy_label = label
        elif label == 2:  # Check if the label is 'deer'
            if random.random() < epsilon:
                noisy_label = 7  # Flip 'deer' to 'horse'
            else:
                noisy_label = label
        elif label == 3:  # Check if the label is 'cat'
            if random.random() < epsilon:
                noisy_label = 5  # Flip 'cat' to 'dog'
            else:
                noisy_label = label
        elif label == 5:  # Check if the label is 'dog'
            if random.random() < epsilon:
                noisy_label = 3  # Flip 'dog' to 'cat'
            else:
                noisy_label = label
        elif label == 7:  # Check if the label is 'truck'
            if random.random() < epsilon:
                noisy_label = 1  # Flip 'truck' to 'automobile'
            else:
                noisy_label = label
        else:
            noisy_label = label # For other classes, keep the label as is
        noisy_labels.append(noisy_label)

    noisy_labels = torch.tensor(noisy_labels) # Convert list to tensor
    return noisy_labels


#"Model Training and Testing Functions with Label Noise Handling

In [6]:
def train_model(train_loader, model, criterion, optimizer, noise_fn, epsilon):
    model.train()  # Set the model to training mode
    running_loss = 0.0  # Initialize the running loss
    correct = 0  # Count of correctly predicted labels
    total = 0  # Total number of labels

    for data, labels in train_loader:  # Iterate over the training data
        optimizer.zero_grad()  # Reset gradients to zero
        data, labels = data.cuda(), labels.cuda()  # Move data and labels to GPU
        noisy_labels = noise_fn(labels.tolist(), epsilon)  # Apply label noise

        outputs = model(data)  # Forward pass: compute predicted outputs
        loss = criterion(outputs, torch.tensor(noisy_labels).cuda())  # Calculate loss
        loss.backward()  # Backward pass: compute gradient
        optimizer.step()  # Perform a single optimization step

        running_loss += loss.item()  # Update running loss
        _, predicted = outputs.max(1)  # Get the predictions
        total += labels.size(0)  # Update the total number of labels
        correct += predicted.eq(torch.tensor(noisy_labels).cuda()).sum().item()  # Update correct predictions

    train_loss = running_loss / len(train_loader)  # Calculate average loss
    train_accuracy = 100 * correct / total  # Calculate training accuracy
    return train_loss, train_accuracy

def test_model(test_loader, model):
    model.eval()  # Set the model to evaluation mode
    correct = 0  # Count of correctly predicted labels
    total = 0  # Total number of labels

    with torch.no_grad():  # Disable gradient calculations
        for data, labels in test_loader:  # Iterate over the test data
            data, labels = data.cuda(), labels.cuda()  # Move data and labels to GPU
            outputs = model(data)  # Forward pass: compute predicted outputs
            _, predicted = outputs.max(1)  # Get the predictions
            total += labels.size(0)  # Update the total number of labels
            correct += predicted.eq(labels).sum().item()  # Update correct predictions

    test_accuracy = 100 * correct / total  # Calculate test accuracy
    return test_accuracy


#Training and Testing Loop for Model with Symmetric and Asymmetric Label Noise"

In [ ]:
import time

for _ in range(1):
    # Training loop for symmetric and asymmetric noise levels
    for epsilon in noise_levels:
       # Initialize data loaders for training and testing
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=256, shuffle=True)
        test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=256, shuffle=False)

        # Symmetric Noise Training
        model = timm.create_model('tf_efficientnet_b3', pretrained=True, num_classes=10).cuda()

        # Set up the loss function and optimizer
        criterion = SCELoss(alpha=1,beta=1,num_classes=10)
        optimizer = optim.Adam(model.parameters(), lr=0.002)
        epochs = 10

        start_train_time = time.time()
        total_train_accuracy_symmetric = 0
        for epoch in range(epochs):
           # Train the model and calculate loss and accuracy for the current epoch
            train_loss, train_accuracy = train_model(train_loader, model, criterion, optimizer, symmetric_label_noise, epsilon)
            total_train_accuracy_symmetric += train_accuracy
        # Record the end time and calculate total training time
        end_train_time = time.time()
        training_time_symmetric = end_train_time - start_train_time

        # Calculate the average training accuracy over all epochs
        average_train_accuracy_symmetric = total_train_accuracy_symmetric / epochs
        # Testing the model
        start_test_time = time.time()
        test_accuracy_symmetric = test_model(test_loader, model)
        end_test_time = time.time()
        inference_time_symmetric = end_test_time - start_test_time # Calculate total inference time

        # Print results for symmetric noise training
        print(f"Symmetric Noise, Epsilon={epsilon}: Average Train Accuracy: {average_train_accuracy_symmetric:.2f}%, Test Accuracy: {test_accuracy_symmetric:.2f}%, Training Time: {training_time_symmetric:.2f}s, Inference Time: {(inference_time_symmetric/1000):.5f}s")

        # Asymmetric Noise Training
        model = timm.create_model('tf_efficientnet_b3', pretrained=True, num_classes=10).cuda()
        # Set up the loss function and optimizer
        criterion = SCELoss(alpha=1,beta=1,num_classes=10)
        optimizer = optim.Adam(model.parameters(), lr=0.002)

        start_train_time = time.time()
        total_train_accuracy_asymmetric = 0
        for epoch in range(epochs):
            train_loss, train_accuracy = train_model(train_loader, model, criterion, optimizer, asymmetric_label_noise, epsilon)
            total_train_accuracy_asymmetric += train_accuracy
        end_train_time = time.time()
        training_time_asymmetric = end_train_time - start_train_time
        # Calculate the average training accuracy over all epochs
        average_train_accuracy_asymmetric = total_train_accuracy_asymmetric / epochs
        # Testing the model
        start_test_time = time.time()
        test_accuracy_asymmetric = test_model(test_loader, model)
        end_test_time = time.time()
        inference_time_asymmetric = end_test_time - start_test_time
        # Print results for Asymmetric noise training
        print(f"Asymmetric Noise, Epsilon={epsilon}: Average Train Accuracy: {average_train_accuracy_asymmetric:.2f}%, Test Accuracy: {test_accuracy_asymmetric:.2f}%, Training Time: {training_time_asymmetric:.2f}s, Inference Time: {(inference_time_symmetric/1000):.5f}s")


Symmetric Noise, Epsilon=0.1: Average Train Accuracy: 76.20%, Test Accuracy: 85.04%, Training Time: 306.61s, Inference Time: 0.00488s
Asymmetric Noise, Epsilon=0.1: Average Train Accuracy: 80.90%, Test Accuracy: 85.50%, Training Time: 306.44s, Inference Time: 0.00488s
Symmetric Noise, Epsilon=0.3: Average Train Accuracy: 52.74%, Test Accuracy: 79.72%, Training Time: 305.02s, Inference Time: 0.00483s
Asymmetric Noise, Epsilon=0.3: Average Train Accuracy: 73.19%, Test Accuracy: 83.74%, Training Time: 307.83s, Inference Time: 0.00483s
Symmetric Noise, Epsilon=0.5: Average Train Accuracy: 39.35%, Test Accuracy: 81.14%, Training Time: 306.96s, Inference Time: 0.00488s
Asymmetric Noise, Epsilon=0.5: Average Train Accuracy: 66.10%, Test Accuracy: 61.73%, Training Time: 304.10s, Inference Time: 0.00488s
